In [4]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 400
pd.options.display.max_columns = 400
%load_ext autoreload
%autoreload 2
from loadcreon import LoadCreon

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
path = 'C:\Data\membership14_final.txt'
lc = LoadCreon(path)

## Let's stratify 80/20 train test holdout for future use

In [8]:
from sklearn.cross_validation import train_test_split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(lc.X, lc.y, test_size=0.2, random_state=771, stratify=lc.y)

In [83]:
pn_mask = (y_train == 0) | (y_train == 1)

In [84]:
y_train_pn = y_train[pn_mask]

In [85]:
X_train_pn = X_train[pn_mask.values]

## Quick LASSO to test run-time

In [31]:
from sklearn.preprocessing import MaxAbsScaler

In [86]:
mas = MaxAbsScaler()

In [87]:
X_train_pn_mas = mas.fit_transform(X_train_pn)

In [36]:
from sklearn.linear_model import LogisticRegression

In [88]:
lasso = LogisticRegression(penalty='l1')

In [89]:
%%time
lasso.fit(X_train_pn_mas, y_train_pn)

Wall time: 69.2 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [43]:
X_test_mas = mas.transform(X_test)

In [91]:
pn_test_mask = (y_test == 0) | (y_test == 1)
y_pn_test = y_test[pn_test_mask]
X_test_pn_mas = X_test_mas[pn_test_mask.values]

In [92]:
y_pn_predict = lasso.predict(X_test_pn_mas)

In [93]:
import sklearn.metrics

In [94]:
sklearn.metrics.f1_score(y_true=y_pn_test, y_pred=y_pn_predict)

0.9903381642512078

In [138]:
sklearn.metrics.confusion_matrix(y_true=y_pn_test, y_pred=y_pn_predict)

array([[215,   0],
       [  8, 410]])

In [136]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_true=y_pn_test, y_pred=y_pn_predict).ravel()
tn, fp, fn, tp

(215, 0, 8, 410)

In [140]:
print(sklearn.metrics.classification_report(y_true=y_pn_test, y_pred=y_pn_predict))

             precision    recall  f1-score   support

          0       0.96      1.00      0.98       215
          1       1.00      0.98      0.99       418

avg / total       0.99      0.99      0.99       633



In [98]:
sklearn.metrics.brier_score_loss(y_true=y_pn_test, y_prob=lasso.predict_proba(X_test_pn_mas)[:,1])

0.011899140392177408

In [102]:
X_test_mas_u = X_test_mas[(y_test == -1).values]

In [103]:
y_predict_u = lasso.predict(X_test_mas_u)

In [110]:
pd.Series(y_predict_u).value_counts()

1    87488
0       19
dtype: int64

In [111]:
pd.Series(lasso.predict(mas.transform(X_train))).value_counts()

1    351533
0      1023
dtype: int64

In [112]:
y_train.value_counts()

-1    350024
 1      1670
 0       862
dtype: int64

## TEST semisuperhelper

In [113]:
from semisuperhelper import SemiSupervisedHelper

In [114]:
ssh = SemiSupervisedHelper(y_train)

In [123]:
x, y = ssh.u(X_train)

In [124]:
x.shape, y.shape, y.value_counts()

((350024, 313), (350024,), -1    350024
 dtype: int64)

In [132]:
y = ssh.pn_assume(unlabeled_to_class=0)

In [133]:
y.value_counts()

0    350886
1      1670
dtype: int64